In [1]:
import pandas as pd

In [2]:
excel_file = pd.ExcelFile('oil.xlsx')

# все листы в файле
excel_file.sheet_names

['СЕНТЯБРЬ',
 'ОКТЯБРЬ',
 'НОЯБРЬ',
 'ДЕКАБРЬ',
 'ЯНВАРЬ',
 'ФЕВРАЛЬ',
 'МАРТ',
 'Апрель',
 'МАЙ',
 'Градуировки (не менять!)',
 'Расчет коэф.(не менять)']

In [3]:
# нужные нам листы
sheet_names_list = [
    'ОКТЯБРЬ',
    'НОЯБРЬ',
    'ДЕКАБРЬ',
    'ЯНВАРЬ',
    'ФЕВРАЛЬ',
    'МАРТ',
    'Апрель',
]

In [4]:
# пример одного листа
df = excel_file.parse(sheet_names_list[0])
df.head(2)

,№ п/п,Дата,Время,№ РГС,Температура РГС,уровень мм,подтоварка,объем,"плотность нефти, зависящая от коэфф-нта объемного расшир-я",плотность при заправке текущей температуре,...,содержание воды,мех.примеси,"хлор,сол",нетто принято,нетто сдано,расход за сутки,нарастающее за месяц,Расход нефти для собственных нужд,Unnamed: 21,Unnamed: 22
0,1,2021-09-30,00:00:00,1.0,20.0,1695.0,0.0,16.208,859.600000,859.6,...,0.12,0.0032,0.0014,0.0,0.046309,NaN,NaN,14.0,NaN,NaN
1,2,2021-09-30,00:00:00,2.0,31.0,0.0,0.0,0.000,848.974156,856.8,...,0.14,0.0045,0.0014,0.0,0.011982,NaN,NaN,NaN,NaN,NaN


In [5]:
# имена всех столбцов
all_columns_list = list(enumerate(df.columns))
all_columns_list

[(0, '№ п/п'),
 (1, 'Дата'),
 (2, 'Время'),
 (3, '№ РГС'),
 (4, 'Температура РГС '),
 (5, 'уровень мм'),
 (6, 'подтоварка'),
 (7, 'объем'),
 (8, 'плотность нефти, зависящая от коэфф-нта объемного расшир-я'),
 (9, 'плотность при заправке текущей температуре'),
 (10, 'масса брутто в  текущий момент'),
 (11, 'принятой'),
 (12, 'сданной'),
 (13, 'содержание воды'),
 (14, 'мех.примеси'),
 (15, 'хлор,сол'),
 (16, 'нетто принято'),
 (17, 'нетто сдано'),
 (18, 'расход за сутки'),
 (19, 'нарастающее    за месяц'),
 (20, 'Расход нефти для собственных нужд'),
 (21, 'Unnamed: 21'),
 (22, 'Unnamed: 22')]

In [6]:
# нужные имена столбцов
column_names_list = [
    all_columns_list[1][1],
    all_columns_list[2][1],
    all_columns_list[3][1],
    all_columns_list[4][1],
    all_columns_list[16][1],
    all_columns_list[17][1],
]
column_names_list

['Дата', 'Время', '№ РГС', 'Температура РГС ', 'нетто принято', 'нетто сдано']

In [7]:
# соберем таблицу за все время
months_df_list = []
for sheet_name in sheet_names_list:
    month_df = excel_file.parse(sheet_name)
    month_filter_df = month_df.filter(column_names_list)
    months_df_list.append(month_filter_df)

all_months_df = pd.concat(months_df_list)

In [8]:
# отсортируем Время
all_months_df['Время'] = all_months_df['Время'].apply(lambda x: '12:00' if '12' in str(x) else '00:00')
# удалим пустые строки
all_months_df.dropna(inplace=True)

In [10]:
# статистика средних за день
all_months_df.groupby(['Дата'])[['Температура РГС ', 'нетто принято', 'нетто сдано']].mean()

,Температура РГС,нетто принято,нетто сдано
Дата,,,
2017-01-30,23.50,0.0,0.181142
2017-01-31,26.50,0.0,0.251162
2021-09-30,25.50,0.0,0.029146
2021-10-01,27.50,0.0,0.150569
2021-10-02,27.50,0.0,0.115321
...,...,...,...
2023-01-27,20.25,0.0,0.170828
2023-01-28,20.00,0.0,0.168250
2023-01-29,19.50,0.0,0.163677


In [11]:
# статистика суммы для каждого РГС
all_months_df.groupby(['№ РГС'])[['нетто принято', 'нетто сдано']].sum()

,нетто принято,нетто сдано
№ РГС,,
1.0,47.839331,61.252963
2.0,57.700665,52.574812


In [12]:
# корреляции
all_months_df.corr()

/var/folders/_1/44gtc23j745bjc805bwt__sw0000gn/T/ipykernel_1520/2630991163.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  all_months_df.corr()


,№ РГС,Температура РГС,нетто принято,нетто сдано
№ РГС,1.000000,-0.100150,0.009917,-0.064825
Температура РГС,-0.100150,1.000000,-0.051230,0.452701
нетто принято,0.009917,-0.051230,1.000000,-0.090243
нетто сдано,-0.064825,0.452701,-0.090243,1.000000
